In [1]:
import h5py
import torch
import json
import numpy as np

import os
import sys
sys.path.append("..")
import torch
import json
from pept3 import helper
import sys
import pandas as pd
import bio_helper
from pept3.model import TransPro, PrositFrag
from tqdm import tqdm
from pept3.dataset import FragDataset, IrtDataset
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%load_ext autoreload


In [2]:
config_data = json.load(open("../checkpoints/data.json"))
frag_dir = config_data['frag']
train_val = os.path.join(frag_dir, "traintest_hcd.hdf5")

train_data = FragDataset(train_val, test_file=None, ratio=0.8)

Got 6787933 sequences


In [4]:
def p2s(p):
    return ".".join([str(i.item()) for i in p])
all_train_peptides = set([p2s(i) for i in train_data.tx[0]])
print(len(all_train_peptides))

del train_data

330983


In [19]:
frag_model = "trans"
read_name = "prosit"

print(read_name)
for which in ["sprot_human", "IGC", "sprot_all", "sprot_bacteria_human"]:
    print(which)
    msms_file = f"/data/prosit/figs/figure6/{which}/maxquant/txt/msms.txt"
    raw_dir = f"/data/prosit/figs/figure6/all_raws"
    save_tab = f"/data/prosit/figs/figure6/{which}/percolator/try/{frag_model}"

    t_t = pd.read_csv(f"{save_tab}/{read_name}_target.psms", sep='\t')
    t_t['label'] = 0
    t_d = pd.read_csv(f"{save_tab}/{read_name}_decoy.psms", sep='\t')
    t_d['label'] = 1
    total_table = t_t.append(t_d, ignore_index=True).sort_values("PSMId")

    threshold = 0.01
    for threshold in [0.1, 0.01, 0.001]:
        table = total_table[total_table['q-value'] < threshold]
        peptides = table['proteinIds']
        peptides = np.array([bio_helper.peptide_to_inter(m).reshape(-1) for m in peptides])
        peptides = torch.from_numpy(peptides)
        labels = table['label']
        target_a = (labels == 0).sum()
        decoy_a = (labels == 1).sum()
        decoy_c, target_c = 0, 0
        for p, l in zip(peptides, labels):
            p = p2s(p)
            if l == 1 and p in all_train_peptides:
                decoy_c += 1
            if l == 0 and p in all_train_peptides:
                target_c += 1
        print(f"    FDR@{threshold:5}:  target {target_c}/{target_a}, decoy {decoy_c}/{decoy_a}", end='\t')
        
        table = total_table[total_table['q-value'] > threshold]
        peptides = table['proteinIds']
        peptides = np.array([bio_helper.peptide_to_inter(m).reshape(-1) for m in peptides])
        peptides = torch.from_numpy(peptides)
        labels = table['label']
        target_a = (labels == 0).sum()
        decoy_a = (labels == 1).sum()
        decoy_c, target_c = 0, 0
        for p, l in zip(peptides, labels):
            p = p2s(p)
            if l == 1 and p in all_train_peptides:
                decoy_c += 1
            if l == 0 and p in all_train_peptides:
                target_c += 1
        print(f"Left: target {target_c}/{target_a}, decoy {decoy_c}/{decoy_a}")

for which in ["trypsin", 'chymo', "lysc", "gluc"]:
    print(which)
    save_tab = f"/data/prosit/figs/fig235/{which}/percolator_up/try/{frag_model}"
    msms_file = f"/data/prosit/figs/fig235/{which}/maxquant/combined/txt/msms.txt"
    raw_dir = f"/data/prosit/figs/fig235/{which}/raw"
    t_t = pd.read_csv(f"{save_tab}/{read_name}_target.psms", sep='\t')
    t_t['label'] = 0
    t_d = pd.read_csv(f"{save_tab}/{read_name}_decoy.psms", sep='\t')
    t_d['label'] = 1
    total_table = t_t.append(t_d, ignore_index=True).sort_values("PSMId")

    threshold = 0.01
    for threshold in [0.1, 0.01, 0.001]:
        table = total_table[total_table['q-value'] < threshold]

        peptides = table['proteinIds']
        peptides = np.array(
            [bio_helper.peptide_to_inter(m).reshape(-1) for m in peptides])
        peptides = torch.from_numpy(peptides)
        labels = table['label']
        target_a = (labels == 0).sum()
        decoy_a = (labels == 1).sum()
        decoy_c, target_c = 0, 0
        for p, l in zip(peptides, labels):
            p = p2s(p)
            if l == 1 and p in all_train_peptides:
                decoy_c += 1
            if l == 0 and p in all_train_peptides:
                target_c += 1
        print(
            f"    FDR@{threshold:5}:  target {target_c}/{target_a}, decoy {decoy_c}/{decoy_a}", end='\t')

        table = total_table[total_table['q-value'] > threshold]
        peptides = table['proteinIds']
        peptides = np.array(
            [bio_helper.peptide_to_inter(m).reshape(-1) for m in peptides])
        peptides = torch.from_numpy(peptides)
        labels = table['label']
        target_a = (labels == 0).sum()
        decoy_a = (labels == 1).sum()
        decoy_c, target_c = 0, 0
        for p, l in zip(peptides, labels):
            p = p2s(p)
            if l == 1 and p in all_train_peptides:
                decoy_c += 1
            if l == 0 and p in all_train_peptides:
                target_c += 1
        print(f"Left: target {target_c}/{target_a}, decoy {decoy_c}/{decoy_a}")


prosit
sprot_human
    FDR@  0.1:  target 5699/19233, decoy 0/1922	Left: target 25360/156205, decoy 2/160488
    FDR@ 0.01:  target 4489/15213, decoy 0/151	Left: target 26570/160225, decoy 2/162259
    FDR@0.001:  target 3519/12517, decoy 0/11	Left: target 27540/162921, decoy 2/162399
IGC
    FDR@  0.1:  target 5036/122034, decoy 0/12202	Left: target 172/106098, decoy 0/101699
    FDR@ 0.01:  target 4411/98115, decoy 0/980	Left: target 797/130017, decoy 0/112921
    FDR@0.001:  target 3343/79212, decoy 0/78	Left: target 1865/148920, decoy 0/113823
sprot_all
    FDR@  0.1:  target 5074/43127, decoy 0/4311	Left: target 2491/144284, decoy 0/142484
    FDR@ 0.01:  target 4063/32015, decoy 0/319	Left: target 3502/155396, decoy 0/146476
    FDR@0.001:  target 3103/25193, decoy 0/24	Left: target 4462/162218, decoy 0/146771
sprot_bacteria_human
    FDR@  0.1:  target 5257/40662, decoy 0/4065	Left: target 4499/142968, decoy 0/141892
    FDR@ 0.01:  target 4225/30519, decoy 0/304	Left: target 55

In [14]:
decoy_c, target_c = 0, 0
for p, l in zip(peptides, labels):
    p = p2s(p)
    if l==1 and p in all_train_peptides:
        decoy_c += 1
    if l==0 and p in all_train_peptides:
        target_c += 1

In [16]:
target_c

0